# Exporting models from Caffe2 to ONNX

In this tutorial we are going to show you how to export a Caffe2 model to ONNX. You can either

- Convert a Caffe2 model to an ONNX model in Python

or 

- Convert a Caffe2 model file to an ONNX model file in the shell

We are going to use the squeezenet model in Caffe2 model zoo, its model files can be downloaded by running:

```shell
$ python -m caffe2.python.models.download squeezenet
```

### Installation

The only thing you need for exporting Caffe2 models to ONNX is the python package `onnx-caffe2`:

```shell
$ pip install onnx-caffe2
```

### Note

In ONNX, the type and shape of the inputs and outpus are required to be presented in the model, while in Caffe2, they are not stored in the model files. So when doing the convertion, we need to provide these extra information to onnx-caffe2 (through a dictionary in Python/a json file in the shell interface).

### Exporting in Python

In [0]:
import onnx
import onnx_caffe2.frontend
from caffe2.proto import caffe2_pb2

# We need to provide type and shape of the model inputs, 
# see above Note section for explanation
data_type = onnx.TensorProto.FLOAT
data_shape = (1, 3, 224, 224)
value_info = {
    'data': (data_type, data_shape)
}

predict_net = caffe2_pb2.NetDef()
with open('predict_net.pb', 'rb') as f:
    predict_net.ParseFromString(f.read())

init_net = caffe2_pb2.NetDef()
with open('init_net.pb', 'rb') as f:
    init_net.ParseFromString(f.read())

onnx_model = onnx_caffe2.frontend.caffe2_net_to_onnx_model(
    predict_net,
    init_net,
    value_info,
)

onnx.checker.check_model(onnx_model)

### Exporting in shell

`onnx-caffe2` has bundled a shell command `convert-caffe2-to-onnx` for exporting Caffe2 model file to ONNX model file.

```shell

$ convert-caffe2-to-onnx predict_net.pb --caffe2-init-net init_net.pb --value-info '{"data": [1, [1, 3, 224, 224]]}' -o sqeezenet.onnx

```

Regarding to the `--value-info` flag, see above Note section for explanation.